# DM assignment 1 - Finding similar items

Konrad Grudzinski - kjgr@kth.se \\
William Berg - willb@kth.se

This notebook implements a technique of finding textually similar documents using shingling, minhashing, Jaccard similarity, and locality-sensitive hashing (LSH). It is implemented in Python, the documents are downloaded using Linux commands directly from the notebook.

It can be run directly as is without any parameters in Google Colab, and probably also on a local installation.

## Download large text files

In this section, various books are downloaded as plain text files from [Project Gutenberg](https://www.gutenberg.org/). A test file called "shingling_test.txt" is created from the first few paragraphs of "Alice in Wonderland".

In [ ]:
!mkdir text_files

In [ ]:
!curl https://www.gutenberg.org/cache/epub/84/pg84.txt > text_files/frankenstein.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  438k  100  438k    0     0  22351      0  0:00:20  0:00:20 --:--:--  112k


In [ ]:
!curl https://www.gutenberg.org/cache/epub/64317/pg64317.txt > text_files/thegreatgatsby.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  299k  100  299k    0     0  15647      0  0:00:19  0:00:19 --:--:-- 88454


In [ ]:
!curl https://www.gutenberg.org/cache/epub/11/pg11.txt > text_files/aliceinwonderland.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  170k  100  170k    0     0   5872      0  0:00:29  0:00:29 --:--:-- 40793


In [ ]:
!curl https://www.gutenberg.org/cache/epub/345/pg345.txt > text_files/dracula.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  870k  100  870k    0     0  57117      0  0:00:15  0:00:15 --:--:--  259k


In [ ]:
!curl https://www.gutenberg.org/cache/epub/174/pg174.txt > text_files/thepictureofdoriangray.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  454k  100  454k    0     0  22545      0  0:00:20  0:00:20 --:--:--  100k


In [ ]:
!curl https://www.gutenberg.org/cache/epub/5200/pg5200.txt > text_files/metamorphosis.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  138k  100  138k    0     0   6279      0  0:00:22  0:00:22 --:--:-- 40898


In [ ]:
!curl https://www.gutenberg.org/cache/epub/1661/pg1661.txt > text_files/sherlockholmes.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  593k  100  593k    0     0  23090      0  0:00:26  0:00:26 --:--:--  190k


In [ ]:
!curl https://www.gutenberg.org/cache/epub/43/pg43.txt > text_files/drjekyllandmrhyde.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  159k  100  159k    0     0  10437      0  0:00:15  0:00:15 --:--:-- 36723


In [ ]:
!curl https://www.gutenberg.org/cache/epub/1232/pg1232.txt > text_files/theprince.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  300k  100  300k    0     0  18862      0  0:00:16  0:00:16 --:--:-- 73784


In [ ]:
!curl https://www.gutenberg.org/cache/epub/1342/pg1342.txt > text_files/priceandprejudice.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  754k  100  754k    0     0  23935      0  0:00:32  0:00:32 --:--:--  182k


In [ ]:
# the project gutenberg server often times out, so I downloaded the files to avoid waiting every time the notebook is openend
#!zip text_files.zip text_files/*
#!unzip text_files.zip

Archive:  text_files.zip
   creating: text_files/
  inflating: text_files/aliceinwonderland.txt  
  inflating: text_files/dracula.txt  
  inflating: text_files/drjekyllandmrhyde.txt  
  inflating: text_files/frankenstein.txt  
  inflating: text_files/metamorphosis.txt  
  inflating: text_files/priceandprejudice.txt  
  inflating: text_files/sherlockholmes.txt  
  inflating: text_files/thegreatgatsby.txt  
  inflating: text_files/thepictureofdoriangray.txt  
  inflating: text_files/theprince.txt  


In [ ]:
!sed -n '58,88p;89q' text_files/aliceinwonderland.txt > shingling_test.txt

In [ ]:
text_files = [
    "text_files/aliceinwonderland.txt",
    "text_files/dracula.txt",
    "text_files/drjekyllandmrhyde.txt",
    "text_files/frankenstein.txt",
    "text_files/metamorphosis.txt",
    "text_files/priceandprejudice.txt",
    "text_files/sherlockholmes.txt",
    "text_files/thegreatgatsby.txt",
    "text_files/thepictureofdoriangray.txt",
    "text_files/theprince.txt"
]

In [ ]:
import hashlib
import numpy as np
import random
import os
import time

## Shingling

In [ ]:
class Shingling:

  def __init__(self, filename, shingle_length, hash_function):
    self.filename = filename
    self.shingle_length = shingle_length
    self.hash_function = hash_function

  # read the file, and clean the text
  def _read_file(self):
    with open(self.filename, "r") as f:
      lines = f.readlines()
    # we remove all white space and some markdown formatting
    whole_text = "".join(map(lambda s: s.replace(" ", "").replace("_", ""), lines))
    return whole_text

  # Takes the shingles, hashes them, and places them in a set
  def _hash_shingles(self, shingles):
    # send each shingle through the provided hash function
    return set(map(self.hash_function, shingles))

  # store the text in a string and place the k-shingles in a set
  def _create_shingles(self):
    s = self._read_file()
    shingles = set()
    # move along the string picking out shingles
    for i in range(0, len(s) - self.shingle_length + 1):
      shingles.add(s[i:i + self.shingle_length])
    return shingles

  # create a set of hashed shingles
  def create_hashed_shingles(self):
    shingles = self._create_shingles()
    return self._hash_shingles(shingles)

In [ ]:
# defines a hash function that hashes the string and cuts it down to a 4 byte integer
hash_function = lambda string: int(hashlib.sha1(str(string).encode("utf-8")).hexdigest(), 16) % (2 ** 32)

#### Shingling test
Create a shingling class and use it to create a set of hashed shingles from a document. 

In [ ]:
shingling_test = Shingling("shingling_test.txt", 10, hash_function)

In [ ]:
test_shingles = shingling_test.create_hashed_shingles()

In [ ]:
len(test_shingles)

1348

## CompareSets

In [ ]:
class CompareSets:
  # takes two sets and computes their jaccard similarity
  def compute_jaccard_similarity(self, set1, set2):
    return len(set1.intersection(set2)) / len(set1.union(set2))

#### CompareSets test
Create a CompareSets class and use it to calculate the similarity of two sets. 

In [ ]:
compare_sets_test = CompareSets()

In [ ]:
compare_sets_test.compute_jaccard_similarity({1,2,3,4,5}, {3,4,5,56,6}) # 3 out of 7 = 0.42... are in both sets

0.42857142857142855

## MinHashing

In [ ]:
class MinHash:

  def __init__(self):
	# dict to store the shingles of each document
    self.raw_hashes_dict = dict()

  # add a document (represented by its shingles set) to the dict
  def add_document(self, document_name, shingles):
    self.raw_hashes_dict[document_name] = shingles

  def remove_document(self, document_name):
    del self.raw_hashes_dict[document_name]

  def _get_all_shingles(self):
    all_shingles = set()
    # get all unique shingles from all documents
    for shingles in self.raw_hashes_dict.values():
      all_shingles = all_shingles.union(shingles)
    return all_shingles

  # used to compute the minhash signature from the shingles of a document
  def _compute_signature(self, permutation_array, shingle_set):
    counter = 0
    # count how many rows it takes in the permuted matrix until the set contains a shingle
    for i in permutation_array:
      if i in shingle_set:
        return counter
      else:
        counter += 1

  def create_signature_matrix(self, length):
    all_shingles = self._get_all_shingles()
    # the signature matrix is sparse and thus represented as a list of indices
	# each key in the dict is a document name, value is an empty list
    signature_dict = {d:[] for d in self.raw_hashes_dict.keys()}
    for i in range(length):
      permutation = np.random.permutation(list(all_shingles))
      for document_name, shingles in self.raw_hashes_dict.items():
        signature_dict[document_name].append(self._compute_signature(permutation, shingles))
    return signature_dict

#### MinHashing test

In [ ]:
text_file_shingles_dict = dict()
for file_name in text_files:
  f_shingling = Shingling(file_name, 10, hash_function)
  f_shingles = f_shingling.create_hashed_shingles()
  text_file_shingles_dict[file_name] = f_shingles

In [ ]:
test_shingles_dict = {k:set(list(v)[:random.randrange(10, 20)]) for k,v in text_file_shingles_dict.items()}

In [ ]:
min_hash_test = MinHash()

In [ ]:
for k,v in test_shingles_dict.items():
  min_hash_test.add_document(k, v)

In [ ]:
test_signature_matrix = min_hash_test.create_signature_matrix(10)
test_signature_matrix

{'text_files/aliceinwonderland.txt': [9, 10, 0, 15, 6, 4, 4, 6, 8, 4],
 'text_files/dracula.txt': [1, 4, 21, 4, 2, 7, 2, 7, 13, 25],
 'text_files/drjekyllandmrhyde.txt': [3, 10, 12, 2, 16, 3, 14, 1, 15, 0],
 'text_files/frankenstein.txt': [11, 2, 3, 7, 3, 7, 1, 5, 1, 3],
 'text_files/metamorphosis.txt': [0, 8, 1, 6, 16, 7, 0, 0, 4, 1],
 'text_files/priceandprejudice.txt': [12, 14, 15, 1, 1, 1, 23, 13, 0, 2],
 'text_files/sherlockholmes.txt': [5, 3, 22, 5, 0, 0, 9, 4, 0, 5],
 'text_files/thegreatgatsby.txt': [6, 0, 4, 0, 8, 5, 20, 2, 10, 15],
 'text_files/thepictureofdoriangray.txt': [8, 1, 2, 8, 3, 7, 3, 5, 2, 29],
 'text_files/theprince.txt': [4, 4, 10, 9, 14, 2, 6, 3, 7, 19]}

## CompareSignatures

In [ ]:
class CompareSignatures:

  def compare_signatures(self, v1, v2):
    assert len(v1) == len(v2)
    return np.sum(np.array(v1) == np.array(v2)) / len(v1)

#### CompareSignatures test

In [ ]:
compare_signatures_test = CompareSignatures()

In [ ]:
compare_signatures_test.compare_signatures([1,2,3,4,5], [1,4,3,7,9])

0.4

## LSH

In [ ]:
class LSH:

  def __init__(self, similarity_threshold, number_of_bands, hash_function):
    self.t = similarity_threshold
    self.b = number_of_bands
    self.h = hash_function

  # generate all pairs of names from a list
  def _generate_pairs(self, l):
    o = []
    for i in range(len(l)):
      for j in range(i+1, len(l)):
        o.append((l[i], l[j]))
    return o

  def _hash_bands_into_buckets(self, signature_matrix_dict, start_index, end_index):
    buckets_dict = dict()
	# iterate over all documents
    for k,v in signature_matrix_dict.items():
	  # extract and hash a band from the document's signature
      band = v[start_index:end_index]
      hash = self.h(list(band))
	  # store the document name in the bucket
      if hash in buckets_dict:
        buckets_dict[hash].append(k)
      else:
        buckets_dict[hash] = [k]
    return buckets_dict

  def generate_candidate_pairs(self, signature_matrix_dict):
	# preparation for the loop
    keys = sorted(list(signature_matrix_dict.keys()))
    first_key = keys[0]
    matrix_length = len(signature_matrix_dict[first_key])
    r = matrix_length // self.b
    common_buckets_counter_dict = dict()
	# iterate over all bands
    for b in range(self.b):
      # calculate start and end index for band
      i = r * b
      j = r * (b + 1)
      if matrix_length - j - r <= 0:
        j = matrix_length
      # get buckets of names for the current band
      buckets_dict = self._hash_bands_into_buckets(signature_matrix_dict, i, j)
      for bucket in buckets_dict.values():
        # buckets with 2 or more documents are candidates, they have at least one band that hashes to the same value
        if len(bucket) >= 2:
          candidate_pairs = self._generate_pairs(sorted(bucket))
          for p in candidate_pairs:
            if p in common_buckets_counter_dict:
              common_buckets_counter_dict[p] += 1
            else:
              common_buckets_counter_dict[p] = 1
    # normalise counts to fractions
    for k,v in common_buckets_counter_dict.items():
      common_buckets_counter_dict[k] = v / self.b
    # only return the pairs with their similarity fraction above a certain threshold
    return {p:v for p,v in common_buckets_counter_dict.items() if v >= self.t}

#### LSH test

In [ ]:
k_hash_function_test = lambda x: hash_function(x) % 257

In [ ]:
lsh_test = LSH(0.1, 5, k_hash_function_test)

In [ ]:
lsh_test.generate_candidate_pairs(test_signature_matrix)

{('text_files/frankenstein.txt', 'text_files/priceandprejudice.txt'): 0.2,
 ('text_files/thepictureofdoriangray.txt', 'text_files/theprince.txt'): 0.2}

## Evaluation

The class below allows to find documents in a corpus of text files stored in a directory.

The similarity threshold for the evaluation is set to 0 to firstly show the computed similarity value for all 10 documents in the corpus, and secondly owed to the size difference between test file and corpus files, resulting in very low similarity even for straight out excerpts from corpus files, as used here. With smaller documents the threshold could be and would be necessary to be set much higher.

In [ ]:
class FindSimilarDocuments:

  def __init__(self, corpus_directory, shingle_length, similarity_threshold, hash_function):
    self.corpus_directory = corpus_directory
    self.shingle_length = shingle_length
    self.similarity_threshold = similarity_threshold
    self.hash_function = hash_function

    # find all files in the corpus and compute their shingles
    file_shingles_dict = dict()
    for f in os.listdir(corpus_directory):
      file_name = os.path.join(corpus_directory, f)
      f_shingling = Shingling(file_name, shingle_length, hash_function)
      f_shingles = f_shingling.create_hashed_shingles()
      file_shingles_dict[f] = f_shingles
    self.file_shingles_dict = file_shingles_dict

    # add the corpus files to the MinHashing class for later
    min_hash = MinHash()
    for k,v in file_shingles_dict.items():
      min_hash.add_document(k, v)
    self.min_hash = min_hash

    self.k_hash_function = lambda x: hash_function(x) % 257

  def _create_shingles_from_file(self, path):
    shingling = Shingling(path, self.shingle_length, self.hash_function)
    shingles = shingling.create_hashed_shingles()
    return shingles

  def find_similar_documents_jaccard(self, document_path, similarity_t=None):
    if similarity_t is None:
      similarity_t = self.similarity_threshold
    # create shingles for input document
    shingles = self._create_shingles_from_file(document_path)
    results = []
    compare_sets = CompareSets()
    # compare with corpus documents using Jaccard similarity of the shingles
    for k,v in self.file_shingles_dict.items():
      similarity = compare_sets.compute_jaccard_similarity(shingles, v)
      if similarity >= similarity_t:
        results.append((k, similarity))
    return results

  def find_similar_documents_lsh(self, document_path, number_of_bands=5, signature_matrix_length=30):
    # create shingles for document
    shingles = self._create_shingles_from_file(document_path)
    # compute signature matrix for corpus and new document
    self.min_hash.add_document(document_path, shingles)
    start = time.time()
    signature_matrix = self.min_hash.create_signature_matrix(signature_matrix_length)
    end = time.time()
    print(f"Create signature matrix using MinHashing: {end - start:.5f}s")
    self.min_hash.remove_document(document_path)
    # find similar documents using LSH
    lsh = LSH(self.similarity_threshold, number_of_bands, self.k_hash_function)
    start = time.time()
    candidate_pairs = lsh.generate_candidate_pairs(signature_matrix)
    end = time.time()
    print(f"Find candidates with LSH: {end - start:.5f}s")
    results = []
    # only return the "similar" document, not document_path, as we know that one
    for (a, b), v in candidate_pairs.items():
      if document_path == a or document_path == b:
        results.append((a if document_path == b else b, v))
    return results

#### Evaluation program tests

In [ ]:
fsd = FindSimilarDocuments(corpus_directory="text_files",
                           shingle_length=10,
                           similarity_threshold=0,
                           hash_function=hash_function)

In [ ]:
fsd.find_similar_documents_jaccard("shingling_test.txt")

[('aliceinwonderland.txt', 0.011350145244811182),
 ('metamorphosis.txt', 0.000523082190478002),
 ('priceandprejudice.txt', 0.0004067107269954245),
 ('thepictureofdoriangray.txt', 0.0003285769333018696),
 ('sherlockholmes.txt', 0.0002934367969743406),
 ('theprince.txt', 0.00021890049408968665),
 ('thegreatgatsby.txt', 0.0003799731783638802),
 ('frankenstein.txt', 0.00026455185516988436),
 ('drjekyllandmrhyde.txt', 0.0003795464823308999),
 ('dracula.txt', 0.0002163271751144797)]

Since the test file was generated by cutting out a part of "Alice in Wonderland", we would expect a similarity search to find that as the most similar book by a significant margin. And indeed, it is considered 20-50 times more similar to its source book than to any other, confirming our expectations. Of course, the Jaccard similarity is still very low, as the excerpt is significantly shorter than the original text. 

In [ ]:
fsd.find_similar_documents_lsh("text_files/metamorphosis.txt",
                               number_of_bands=5,
                               signature_matrix_length=30)

Create signature matrix using MinHashing: 79.59212s
Find candidates with LSH: 0.00027s


[('metamorphosis.txt', 1.0)]

In [ ]:
fsd.find_similar_documents_lsh("shingling_test.txt",
                               number_of_bands=5,
                               signature_matrix_length=50)

Create signature matrix using MinHashing: 132.35803s
Find candidates with LSH: 0.00049s


[('aliceinwonderland.txt', 0.2)]

Using LSH for similarity search, we try whether it can find a perfect match, and also the excerpt from a book. It comes as no surprise that inputting a document from its corpus has a 100% match with one of the documents from the corpus. Furthermore, it was able to find the book the test excerpt was taken from.

Computing Jaccard similarity and LHS are both very fast, however, the  problem with this implementation is the recomputation of the whole signature matrix for every input, which is necessary here, as the shingles of the input are not known in advance. This is the most time consuming procedure of the LSH process; LSH requires these signatures while Jaccard doesn't. This could be avoided by saving the permutations on the whole shingle-by-document matrix and applying them only to the new input, thus saving a lot of time. When the signature matrix is known, our results indicate that the actual process of finding similar items is faster when using LSH as opposed to computing the Jaccard similarities directly. It should also be noted that the benefits of using LSH are more obvious when the corpus is very large, as the number of candidate pairs will probably be much lower than the number of pairs overall in that case. It is also worth discussing the impact that the number of bands has on the effectiveness of LSH. If the algorithm has a hard time finding candidate pairs, it might be a good idea to increase the number of bands. Indeed, while testing our code, it was sometimes hard to find any candidates at all when this number remained low. This is of course a trade-off between speed and effectiveness as a higher number of bands will negatively impact the runtime of the algorithm. 